<a href="https://colab.research.google.com/github/jufabeck2202/KI-Lab/blob/main/%5BKI_Lab2021%5D_Assignmanet_5_OWCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 5: 
Text Preparation:
1. We use the same dataset from last time: https://github.com/tblock/10kGNAD.
2. Download the files train.csv and test.csv
3.Filter both files to include only lines that fulfill both of the following conditions:
    1. The line is of the category „Wirtschaft“ or „Sport“.
    2. The line only contains valid characters (note that ‘\‘ is a line break in Python):
4. The number of valid lines for training and test should be 2154 and 244.

5. Divide the valid training data lines into tokens using SpaCy.

---------------------------------------------

6. Determine the 5000 most common tokens in the training data.
7. For those tokens, create the word_to_ix dictionary as shown in the tutorial.
8. Train a language classification model using the two categories and the sentences using an LSTM model

9. Use the same architecture as in the tutorial but an embedding dim of 128 and a hidden dim of 256.
10. In one training loop go through every line of the training data and update the weights after every line (like in the tutorial).
11. For every line, transform the words according to the word_to_ix dictionary and feed it to the LSTM as one sequence. Note: Skip words that are not in word_to_ix (they are not part of the 5000 most common tokens.)
12. Do this for 5 epochs and print the accuracy on the test data after every epoch.
13. Note: Since we do not train in batches, but line per line, this will take quite a while (~ 15 minutes for one training loop).

#Bonus
14. The training is quite slow, since we do not train in batches but line by line. Providing the text lines in batches to the LSTM, will not work out of the box, because all tensors in one batch need to have the same length.
15. The DataLoader class can be created with a collate_fn function that can be used to bring all lines in one batch to the same size:
16. Determine the longest tensor in the current batch.
17. Fill up all tensors that are shorter with an unused padding value (e.g. -1).
18. Task: Put the data into a DataLoader, pad the lines in one batch to the same length and train in batches on the GPU. Compare the training time to the line-wise training.

#Import

In [86]:
!pip install -U spacy[cuda100] de &> /dev/null
!python -m spacy download de &> /dev/null
!pip install -q "tqdm>=4.36.1" &> /dev/null

In [87]:
import pandas as pd
import torch
import spacy
spacy.prefer_gpu()
nlp_ger = spacy.load("de")
from tqdm import tqdm
import string
import torch
import torch.nn as nn
import random
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from collections import Counter
from tabulate import tabulate
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


# Import files


In [88]:
df_test = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv', \
                      names=['trash','category','data'], sep='^([^;]+);', engine="python")
df_test = df_test.drop(columns=['trash'])
#df_test

In [89]:
df_train = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv', \
                      names=['trash','category','data'], sep='^([^;]+);', engine="python")
df_train = df_train.drop(columns=['trash'])

#df_train

In [90]:
#filter:
df_test = df_test.loc[(df_test["category"]=="Wirtschaft") | (df_test["category"]=="Sport")]
df_train = df_train.loc[(df_train["category"]=="Wirtschaft") | (df_train["category"]=="Sport")]

print(len(df_test))
print(len(df_train))

261
2351


In [91]:
#rows_test = []
#for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0], position=0, leave=True):
#    rows_test.append(row.Data)

#rows_train = []
#for index, row in tqdm(df_train.iterrows(), total=df_train.shape[0], position=0, leave=True):
#    rows_train.append(row.Data)


In [92]:
all_letters = string.ascii_letters + 'ÄÖÜäöüß–' + string.punctuation + \
                string.digits + string.whitespace

def check(line):
    return all(ch in list(all_letters) for ch in line)

df_test_1 = df_test[df_test['data'].apply(lambda line : all(ch in list(all_letters) for ch in line))]
print(len(df_test_1))

df_train_1 = df_train[df_train['data'].apply(lambda line : all(ch in list(all_letters) for ch in line))]
print(len(df_train_1))

244
2154


In [93]:
rows_train_token = []
for sentence in tqdm(df_train_1['data'], position=0, leave=True):
    token = nlp_ger(sentence)
    for token_s in token:
        rows_train_token.append(token_s.text)


100%|██████████| 2154/2154 [01:18<00:00, 27.49it/s]


In [94]:
word_freq = Counter(rows_train_token)
common_words = word_freq.most_common(5000)
print(common_words)

[('.', 45468), (',', 40543), ('der', 22639), ('die', 19536), ('in', 13411), ('und', 12308), ('(', 9056), ('den', 8954), (')', 8914), ('von', 7050), ('mit', 6765), ('–', 6456), ('im', 6280), ('auf', 6082), ('das', 5982), ('zu', 5699), (':', 5650), ('für', 5497), ('ist', 5447), ('Die', 5371), ('nicht', 5366), ('sich', 4838), ('dem', 4559), ('ein', 4363), ('es', 4280), (' ', 4049), ('am', 3868), ('des', 3752), ('eine', 3727), ('auch', 3704), ('nach', 3453), ('bei', 3393), ('als', 3367), ('Der', 3350), ('hat', 3176), ('an', 3002), ('dass', 2954), ('aus', 2710), ('war', 2606), ('aber', 2550), ('werden', 2490), ('einem', 2479), ('noch', 2426), ('gegen', 2413), ('um', 2408), ('vor', 2356), ('Das', 2327), ('einen', 2254), ('haben', 2204), ('Prozent', 2176), ('wird', 2169), ('sind', 2109), ('er', 2052), ('Euro', 2049), ('wie', 2046), ('einer', 2001), ('nur', 1868), ('mehr', 1840), ('über', 1727), ('hatte', 1677), ('zum', 1629), ('sie', 1591), ('/', 1493), ('so', 1475), ('In', 1469), ('man', 137

In [95]:
word_to_ix = {}

for word in common_words:
    if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

{('.', 45468): 0, (',', 40543): 1, ('der', 22639): 2, ('die', 19536): 3, ('in', 13411): 4, ('und', 12308): 5, ('(', 9056): 6, ('den', 8954): 7, (')', 8914): 8, ('von', 7050): 9, ('mit', 6765): 10, ('–', 6456): 11, ('im', 6280): 12, ('auf', 6082): 13, ('das', 5982): 14, ('zu', 5699): 15, (':', 5650): 16, ('für', 5497): 17, ('ist', 5447): 18, ('Die', 5371): 19, ('nicht', 5366): 20, ('sich', 4838): 21, ('dem', 4559): 22, ('ein', 4363): 23, ('es', 4280): 24, (' ', 4049): 25, ('am', 3868): 26, ('des', 3752): 27, ('eine', 3727): 28, ('auch', 3704): 29, ('nach', 3453): 30, ('bei', 3393): 31, ('als', 3367): 32, ('Der', 3350): 33, ('hat', 3176): 34, ('an', 3002): 35, ('dass', 2954): 36, ('aus', 2710): 37, ('war', 2606): 38, ('aber', 2550): 39, ('werden', 2490): 40, ('einem', 2479): 41, ('noch', 2426): 42, ('gegen', 2413): 43, ('um', 2408): 44, ('vor', 2356): 45, ('Das', 2327): 46, ('einen', 2254): 47, ('haben', 2204): 48, ('Prozent', 2176): 49, ('wird', 2169): 50, ('sind', 2109): 51, ('er', 205

In [96]:
tag_to_ix = {"Wirtschaft": 0, "Sport": 1}


In [97]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [98]:
print(f"{len(word_to_ix)} {len(tag_to_ix)}")
model = LSTMTagger(128, 256, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

def prepare_sequence(seq, to_ix):
  data = [to_ix[line] for line in seq if line in to_ix ]
  print(data)
  return torch.tensor(data, dtype=torch.long)



# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(df_train["data"][0].split(), word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(5):  # again, normally you would NOT do 300 epochs, it is toy data
    for i, entry in df_train.iterrows():
        sentence = entry["data"].split()
        label = entry["category"]
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence([tags], tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

5000 2
[]


RuntimeError: ignored